In [26]:
pwd

'/content/drive/My Drive'

In [2]:
cd drive/MyDrive/

/content/drive/MyDrive


In [27]:
import tensorflow as tf
from pathlib import Path
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers
import cv2
from tqdm import tqdm
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from tqdm import tqdm
from random import shuffle, choice, uniform
from sklearn.metrics import confusion_matrix

In [28]:
from os.path import isdir, dirname, abspath, join
from os import makedirs
from tensorflow.keras.callbacks import (ModelCheckpoint, TensorBoard, 
                                        EarlyStopping, LearningRateScheduler)
from io import BytesIO
from natsort import natsorted
from tensorflow.keras import backend as K
from tensorflow.keras import Sequential,Model

from tensorflow.keras.applications import (DenseNet201, InceptionV3, MobileNetV2,
                                           ResNet101, Xception, EfficientNetB7,VGG19, NASNetLarge)
from tensorflow.keras.applications import (densenet, inception_v3, mobilenet_v2,
                                           resnet, xception, efficientnet, vgg19, nasnet)

from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.layers.experimental.preprocessing import Rescaling, Resizing
from tensorflow.keras.utils import Progbar

# Define paths and labels

In [29]:
ROOT = '/content/drive/MyDrive'
data_path = 'cropped/'
train_path = data_path + 'train'
val_path = data_path + 'val'

labels = {v:k for k, v in enumerate(listdir(train_path))}
labels

{'black': 11,
 'blue': 1,
 'brown': 2,
 'green': 3,
 'grey': 4,
 'orange': 0,
 'pink': 5,
 'purple': 6,
 'red': 7,
 'silver': 8,
 'white': 9,
 'yellow': 10}

# Generate TFRecords by compressing jpeg files and their corresponding one hot labels

In [6]:
# https://medium.com/ymedialabs-innovation/how-to-use-tfrecord-with-datasets-and-iterators-in-tensorflow-with-code-samples-ffee57d298af

class GenerateTFRecord:
    def __init__(self, path):
        self.path = Path(path)
        self.labels = {v:k for k, v in enumerate(listdir(path))}

    def convert_image_folder(self, tfrecord_file_name):
        # Get all file names of images present in folder
        img_paths = list(self.path.rglob('*.jpg'))
        shuffle(img_paths)
        img_paths = img_paths

        with tf.io.TFRecordWriter(tfrecord_file_name) as writer:
            for img_path in tqdm(img_paths, desc='images converted'):
                example = self._convert_image(img_path)
                writer.write(example.SerializeToString())

    def _convert_image(self, img_path):
        label = self.labels[img_path.parent.stem]
        img_shape = mpimg.imread(img_path).shape
        # print(tf.one_hot(label, depth=len(self.labels), on_value=1, off_value=0))
        # Read image data in terms of bytes
        with tf.io.gfile.GFile(img_path, 'rb') as fid:
            image_data = fid.read()

        example = tf.train.Example(features = tf.train.Features(feature = {
            'image': tf.train.Feature(bytes_list = tf.train.BytesList(value = [image_data])),
            'label': tf.train.Feature(int64_list = tf.train.Int64List(value = tf.one_hot(label, 
                                                                                         depth=len(labels), 
                                                                                         on_value=1, 
                                                                                         off_value=0))),
        }))
        return example


t = GenerateTFRecord(path=train_path)
t.convert_image_folder('orig_data.tfrecord')


images converted: 100%|██████████| 4881/4881 [33:30<00:00,  2.43it/s]


# Defining dict for models and their corresponding blocks for unfreeze during transfer learning

In [30]:
models = {
    'densenet': DenseNet201,
    'xception': Xception,
    'inceptionv3': InceptionV3,
    'effecientnetb7': EfficientNetB7,
    'vgg19': VGG19,
    'nasnetlarge': NASNetLarge,
    'mobilenetv2': MobileNetV2,
    'resnet': ResNet101
}

# models['densenet']()

preprocess_pipeline = {
    'densenet': densenet.preprocess_input,
    'xception': xception.preprocess_input,
    'inceptionv3': inception_v3.preprocess_input,
    'effecientnetb7': efficientnet.preprocess_input,
    'vgg19': vgg19.preprocess_input,
    'nasnetlarge': nasnet.preprocess_input,
    'mobilenetv2': mobilenet_v2.preprocess_input,
    'resnet': resnet.preprocess_input
}


# Utility functions:
- `generator`: Configuring `tf.data` generator.
- `augment`: Augmentation using `tf.image` 90 degree rotation, flip and resize.
- `_parse_function`: Function to parse tfrecords to tensor.
- `configure_for_performance`: Configuring caching mechanism for tf.data
- Sample test for validating code.

In [31]:
model_name = 'xception'
def augment(image, rotate=True, flip_vertical=True, 
            flip_horizontal=True, random_brightness=True):

    image = tf.cast(image, tf.float32)
    image = tf.expand_dims(image, axis=0)
    image = preprocess_pipeline[model_name](image)
    image = tf.squeeze(image, axis=0)

    image = tf.reverse(image, axis=[0]) if flip_vertical else image   # flip top to bottom.
    image = tf.reverse(image, axis=[1]) if flip_horizontal else image # flip left to right.
    # Flip up to bottom can be generated if rotate function given k = 2 in rotate part.

    if rotate:
        image = tf.expand_dims(image, axis=0)
        image = tf.image.rot90(image, k=choice([1,2,3])) # Random rotate
        image = tf.squeeze(image, axis=0)

    if random_brightness:
        image = tf.image.random_brightness(image, max_delta=0.1)

    return image

def _augment_function(image, label):
    flip_vertical = np.random.uniform(0, 1) >= 0.5
    flip_horizontal = np.random.uniform(0, 1) >= 0.5
    rotate = np.random.uniform(0, 1) >= 0.5
    random_brightness = np.random.uniform(0, 1) >= 0.5

    image = augment(image,
                    flip_horizontal=flip_horizontal, 
                    flip_vertical=flip_vertical,
                    rotate=rotate,
                    random_brightness=random_brightness)
    
    # matplotlib wants [0,1] values
    image = tf.clip_by_value(image, clip_value_min=0, clip_value_max=1)
    
    return image, label

def _parse_function(tfrecord):
    img_size = 224
    # Extract features using the keys set during creation
    features = {
        'image': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([12], tf.int64)
    }
    # Extract the data record
    sample = tf.io.parse_single_example(tfrecord, features)
    image = tf.image.decode_jpeg(sample['image'])

    image = tf.image.resize(image, [img_size, img_size], 
                            method=tf.image.ResizeMethod.BILINEAR)
    # augmentation
    label = sample['label']
    return image, label


def configure_for_performance(ds, buffer_size, batch_size):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=buffer_size)
    return ds


def generator(tfrecord_file, batch_size, n_data, validation_ratio, 
              reshuffle_each_iteration=False):
    reader = tf.data.TFRecordDataset(filenames=[tfrecord_file])
    reader.shuffle(n_data, reshuffle_each_iteration=reshuffle_each_iteration)
    AUTOTUNE = tf.data.experimental.AUTOTUNE

    val_size = int(n_data * validation_ratio)
    train_ds = reader.skip(val_size)
    val_ds = reader.take(val_size)

    train_ds = train_ds.map(_parse_function, num_parallel_calls=AUTOTUNE)
    train_ds = train_ds.map(_augment_function, num_parallel_calls=AUTOTUNE)
    train_ds = configure_for_performance(train_ds, AUTOTUNE, batch_size).repeat()

    val_ds = val_ds.map(_parse_function, num_parallel_calls=AUTOTUNE)
    val_ds = val_ds.map(_augment_function, num_parallel_calls=AUTOTUNE)
    val_ds = configure_for_performance(val_ds, AUTOTUNE, batch_size).repeat()
    return train_ds, val_ds

tfrecord_file = 'cropped_data.tfrecord'
validation_ratio = 0.2
# n_data = len(list(Path('original/train').rglob('*.jpg')))
n_data = 50
train, val = generator(tfrecord_file, 2, n_data, validation_ratio, True)

# i = 0
# for x, y in train:
#   print(list(labels.keys())[int(tf.argmax(y[0]))])
#   i+=1
#   if i == 1:
#     break
# x, y = train[0]

# Utility Functions for training:
- `create_model`: creates deep model / freeze or unfreeze layers / adds classification layers.
- `scheduler`: reduces the learning rate over epochs.
- `manage_runs`: organizes paths to record the callbacks.

In [32]:
def create_model(optimizer, name='densenet', include_compile=True, blocks_to_train=[]):
    base_model = models[name](include_top=False, weights='imagenet')
    # model = Model(base_model.inputs, base_model.layers[-1].output)

    if bool(blocks_to_train):
        for block in blocks_to_train:
            for layer in base_model.layers:
                if block in layer.name:
                  layer.trainable = True
                else:
                  layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.layers[-1].output)
    x = Dense(1024, activation='relu')(x)
    output = Dense(12, activation='softmax')(x)

    model = Model(base_model.inputs, output)
    
    if include_compile:
        model.compile(loss='categorical_crossentropy',
                      optimizer=optimizer,
                      metrics=['accuracy'])
    
    return model


def scheduler(epoch):
    # Every 10 epochs, the learning rate is reduced to 1/10 of the original
    if epoch == 10:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)

    if epoch == 100:
        lr = K.get_value(model.optimizer.lr)
        K.set_value(model.optimizer.lr, lr * 0.1)

    # if epoch % 10 == 0 and epoch != 0:
    #     lr = K.get_value(model.optimizer.lr)
    #     K.set_value(model.optimizer.lr, lr * 0.5)
    return K.get_value(model.optimizer.lr)


def generate_path(path_to_output, last_run=False):
    runs = natsorted([path for path in listdir(path_to_output) if path.startswith("run_tf_data")])
    if last_run:
        if not bool(runs):
          path = join(path_to_output, "run_tf_data_1")
        else:
          path = join(path_to_output, runs[-1])

        return path
    if not bool(runs):
        path = join(path_to_output, 'run_tf_data_1')
    else:
        f = runs[-1].rsplit("data_")[1]
        path = join(path_to_output, 'run_tf_data_' + str(int(f) + 1))
    
    return path


def manage_runs(out_path, save_weights=True, reduce_lr_on_epoch=False, 
                reduce_lr_on_plateau=False):
    # get the root path of the project.
    assert isdir(out_path), "Path does not exist."
    # root = dirname(abspath(__file__))
    # Absolute path to output directory.
    out_path = join(ROOT, out_path)
    path_to_save = generate_path(out_path)

    makedirs(path_to_save, exist_ok=True)
    callbacks = []

    if save_weights:
        weights_dir = f'{path_to_save}/weights/'
        makedirs(weights_dir)
        checkpoint = ModelCheckpoint(filepath=weights_dir + '/epoch.{epoch}.loss.{val_loss:.3f}.h5',
                                     save_best_only=True,
                                     monitor='val_loss',
                                     save_weights_only=True)
        callbacks.append(checkpoint)

    if reduce_lr_on_epoch:
        lr_epoch = LearningRateScheduler(scheduler)
        callbacks.append(lr_epoch)

    if reduce_lr_on_plateau:
        lr_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=50, 
                                    min_lr=1e-5)
        callbacks.append(lr_plateau)

    return callbacks

# Tensorboard Utils

In [33]:
def plot_to_image(figure):
    # save plot to png file
    buf = BytesIO()
    plt.savefig(buf, format='png')
    plt.close(figure)

    buf.seek(0)
    image = tf.image.decode_png(buf.getvalue(), channels=4)

    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image

def image_to_grid(data, labels, class_names):

    assert data.ndim == 4
    figure = plt.figure(figsize=(10,10))
    num_images = data.shape[0]
    size = int(np.ceil(np.sqrt(num_images)))
    # class_names[int(tf.argmax(y[0]))
    for i in range(data.shape[0]):
        plt.subplot(size, size, i+1, 
                    title=class_names[int(tf.argmax(labels[i]))])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        
        # Grayscale images
        # if data.shape[3] == 1:
        #     plt.imshow(data[i], cmap=plt.cm.binary)
        # else:
        #     plt.imshow(data[i])

    return figure
    
def get_confusion_matrix(y_labels, logits, class_names):
    preds = np.argmax(logits, axis=1)
    labels = tf.argmax(y_labels, axis=1).numpy()
    cm = confusion_matrix(
        labels,
        preds, 
        labels=np.arange(len(class_names))
    )

    return cm

def plot_confusion_matrix(cm, class_names):
    size=len(class_names)
    figure=plt.figure(figsize=(size, size))
    plt.title("Confusion Matrix")

    indices = np.arange(len(class_names))
    
    plt.xticks(indices, class_names, rotation=45)
    plt.yticks(indices, class_names)
    cm=np.around(
        cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=3
    )

    threshold = cm.max() / 2.0

    for i in range(size):
        for j in range(size):
            color = 'white' if cm[i,j] > threshold else "black"

            plt.text(
                i, j, cm[i, j], horizontalalignment="center", color=color, 
            )

    plt.tight_layout()
    plt.xlabel("True Label")
    plt.ylabel("Prediction label")
    cm_image = plot_to_image(figure)
    
    return  cm_image


# Defining model parameters and generators

In [34]:
"""
########################################################
##################  train parameters  ##################
########################################################
"""

n_data = len(list(Path(data_path).rglob('*.jpg')))
validation_ratio = 0.2
val_size = int(n_data * validation_ratio)
train_size = n_data - val_size
batch_size = 64
n_epochs = 2
n_workers = 5

train_steps = train_size//(batch_size)
val_steps = val_size//(batch_size)


filename = '/content/drive/MyDrive/cropped_data.tfrecord'


train_ds, val_ds = generator(filename,
                            batch_size=batch_size,
                            n_data=n_data,
                            validation_ratio=validation_ratio,
                            reshuffle_each_iteration=True)

# callbacks = manage_runs('runs',
#                         save_weights=False,
#                         reduce_lr_on_epoch=True,
#                         reduce_lr_on_plateau=False)

In [35]:
!rm -r runs/*

# Training process

|model|val loss|val acc|wall time|epochs|generator
|:---:|:---:|:---:|:---:|:---:|:---:|
|densnet 121|1.13|0.6633|1h 21min 2s|300|`tf.data`|

Wall time: 1h 21min 2s

# TensorFlow Custom Loop
----
**Includes:**
- Generate data with `tf.data.Dataset`
- `TFRecord`
- Tensorboard
- - validation data images
- - images of cofusion matrix
- - Train/Validation accuracy and loss
- Transfer Learning
- Callbacks

In [ ]:
""" 
########################################################
##################  configure tensorboard  #############
########################################################
"""

# model_name = choice(list(models.keys()))
model_name = 'xception'
# blocks_to_train = block_trainables[model_name]

path_to_run = generate_path(join(ROOT, 'runs'), last_run=True)

tb_train_path = join(path_to_run, 'logs','train')
tb_test_path = join(path_to_run, 'logs', 'test')

train_writer = tf.summary.create_file_writer(tb_train_path)
test_writer = tf.summary.create_file_writer(tb_test_path)
train_step = test_step = 0

"""
########################################################
##########  configure model inputs and params  #########
########################################################
"""

blocks_to_train = []
lr = 1e-4
optimizer = SGD(lr=lr, decay=1e-6,momentum=0.9,nesterov=True)
# optimizer = Adam(learning_rate=0.0001)
loss_fn = tf.keras.losses.CategoricalCrossentropy(from_logits=True)
acc_metric = tf.keras.metrics.CategoricalCrossentropy()
model = create_model(optimizer, name=model_name, include_compile=False,
                    blocks_to_train=blocks_to_train)

""" 
########################################################
##################  Training Loop  #####################
########################################################
"""

metrics = {'acc': 0.0, 'loss': 0.0, 'val_acc': 0.0, 'val_loss': 0.0, 'lr': lr}

for epoch in range(n_epochs):
    # Iterate through the training set
    print("\epoch {}/{}".format(epoch+1,n_epochs))
    progress_bar = Progbar(train_size, stateful_metrics=list(metrics.keys()))

    train_gen = train_ds.take(train_size//batch_size)

    for batch_idx, (x, y) in enumerate(train_gen):
        with tf.GradientTape() as tape:
            y_pred = model(x, training=True)
            loss = loss_fn(y, y_pred)

        gradients = tape.gradient(loss, model.trainable_weights)
        optimizer.apply_gradients(zip(gradients, model.trainable_weights))
        acc_metric.update_state(y, y_pred)
        train_step += 1
        progress_bar.update(batch_idx*batch_size, values=[('acc',acc_metric.result()), 
                                       ('loss', loss)])
        # print(batch_idx, " / ", len(train_gen))

    with train_writer.as_default():
        tf.summary.scalar("Loss", loss, step=epoch)
        tf.summary.scalar(
            "Accuracy", acc_metric.result(), step=epoch
        )
        
    # reset accuracy between epochs (and for testing and test)
    
    acc_metric.reset_states()

    """ 
    ########################################################
    #####################  Validation  #####################
    ########################################################
    """

    # acc_metric.reset_states()
    val_gen = val_ds.take(val_size//batch_size)
    for batch_idx, (x,y) in enumerate(val_gen):
        confusion = np.zeros((len(labels), len(labels)))
        figure = image_to_grid(x.numpy(),
                               y.numpy(), 
                                class_names=list(labels.keys()))
        y_pred = model(x, 
                       training=False)
        loss = loss_fn(y, y_pred)
        acc_metric.update_state(y, 
                                y_pred)
        confusion += get_confusion_matrix(y, y_pred, class_names=list(labels.keys()))

        with test_writer.as_default():
            tf.summary.scalar("Loss", loss, step=epoch)
            tf.summary.scalar("Accuracy", acc_metric.result(), step=epoch)
            tf.summary.image('Confusion Matrix',
                            plot_confusion_matrix(confusion / batch_idx,
                                                class_names=list(labels.keys())), step=epoch)
            tf.summary.image("Visualize images",plot_to_image(figure), step=epoch)

    progress_bar.update(train_size, values=[('val_acc', acc_metric.result()), ('val_loss', loss)])

    # reset accuracy between epochs (and for testing and test)
    acc_metric.reset_states()